<h1 align="center" id="heading">Testing for Equality in High-dimensional Covariance Matrices</h1>

<br />

This notebook aims to present a discussion on how to test for equality between covariance matrices in high-dimensional settings. Firstly, more classical statistical approaches, such as Box's M-test, were illustrated. Nonetheless, we also presented a discussion on why these traditional statistical techniques introduce challenges and tend to fail in high-dimensional settings. So naturally, we also pointed out some refined methodologies, which exploit recent advances in Random Matrix Theory to overcome the limitations of the classical statistical results.

## Contents:
* [Box's M-test](#box)
* [The Curse of Dimensionality](#curse-dim)
* [Random Matrix Theory and Central Limit Theorems](#random-matrix)
* [Conclusion](#con)

## <a class="anchor" id="box"></a>Box's M-test

Let's assume we have $G$ groups (or sub-populations), and we want to check whether or not $\boldsymbol{\Sigma}_{1} = \dots = \boldsymbol{\Sigma}_{G}$, where $\boldsymbol{\Sigma}_{g}$ denotes the covariance matrix of the group $g$, for $g\in\{1,\dots,\,G\}$.

A standard and simple approach to test for equality between multiple covariance matrices is Box's M-test. Let $n_g$ be the sample size for group $g$ and let $\mathbf{X}_{g}$ be the design matrix (or data matrix) for group $g$, this is

$$ \mathbf{X}_{g} = \begin{pmatrix} \mathbf{x}^{g}_{1} & \mathbf{x}^{g}_{2} & \dots & \mathbf{x}^{g}_{n_{g}} \end{pmatrix}' \in \mathbb{R}^{n_{g}\times p}, \;\;\;\;\;\; g\in\{1,\dots,\,G\},$$

where each $\mathbf{x}^{g}_{i}$, for $i\in\{1,\dots,\,n_{g}\}$, is a $p$-dimensional normally distributed random vector with mean $\boldsymbol{\mu}_{g}\in\mathbb{R}^{p}$ and covariance matrix $\boldsymbol{\Sigma}_{g}\in\mathbb{M}^{p\times p}$, with $\mathbb{M}^{p\times p}$ being the set of all symmetric positive definite matrices of size $p\times p$ , in other words,

$$\mathbf{x}^{g}_{i}\sim\mathcal{N}_{p}\left(\boldsymbol{\mu_g},\, \boldsymbol{\Sigma}_{g}\right),\;\;\;\;\;\; i\in\{1,\dots,\,n_g\},$$

where $p$ is the dimension of the feature space.

Following the notation from Anderson (2003),

* Let $n:=\sum_{g=1}^{G}n_{g}$ be the overall sample size across the $G$ groups
* Let $A_{g} := (n_g - 1)\mathbf{S}_{g}$, where $\mathbf{S}_{g}\in\mathbb{M}^{p\times p}$ is the sample covariance matrix of the group $g$
* Let $A := \sum_{g=1}^{G}A_{g}$

We would like to test

$$H_{0}: \boldsymbol{\Sigma}_{1} = \dots = \boldsymbol{\Sigma}_{G} = \boldsymbol{\Sigma}, $$

against

$$H_{1}: \exists \; \boldsymbol{\Sigma}_{g} \neq \boldsymbol{\Sigma}, \;\;\;\;\;\; g\in\{1,\dots,\,G\}.$$

Following the procedure presented in Anderson (2003), we have that the maximum likelihood ratio test statistic, denoted by $\lambda^{*}_{1}$, is

 $$\lambda^{*}_{1} = \frac{\prod_{g=1}^{G}\left|A_{g}\right|^{\frac{\left(n_g - 1\right)}{2}}}{\left| A\right|^{\frac{\left(n-G\right)}{2}}} \times \frac{(n-G)^{\frac{(n-G)p}{2}}}{\prod_{g=1}^{G}(n_g - 1)^{\frac{(n_g-1)p}{2}}}.$$
 
Taking log we have that

$$ \ln(\lambda_{1}^{*}) = \left\{\sum_{g=1}^{G}\frac{(n_{g}-1)}{2}\ln\left(\left|A_{g}\right|\right)\right\} - \left\{ \frac{(n-G)}{2}\ln\left(\left|A\right|\right)   \right\} + \left\{ \frac{(n-G)p}{2}\ln(n-G)  \right\} - 
\left\{\sum_{g=1}^{G}\frac{p(n_{g}-1)}{2}\ln(n_{g}-1)\right\}. $$

The function ```log.lambda(Ag, ng, G, p, n)``` computes $\ln(\lambda_{1}^{*})$. The argument ```Ag``` is a ```list``` of length $G$, which contains all $A_{g}$ matrices. ```ng``` is a ```vector``` of length $G$ that contains the group sample sizes for each sub-population, ```G``` is the total number of sub-populations, ```p``` is the dimension of the feature space, and ```n``` is the overall sample size.

**Note:**

It is not a secret that for-loops in R can be very slow and computationally inefficient. However, we can exploit the functions ```apply()```, ```lapply()```,  ```sapply()```, and ```Reduce()``` to avoid writing explicit for-loops and increase our code speed.

In [154]:
log.lambda <- 
    function(Ag, ng, G, p, n){
        
        # Arguments:
        # Ag -- list containing each one of the Ag matrices for each sub-population
        # ng -- vector containing each one of the sample sizes for each sub-population
        # G -- total number of sub-populations
        # p -- dimension of the feature space
        # n -- overall sample size
    
        # Returns:
        # log.lambda -- real value
        
        # Obtain matrix A
        A <- Reduce('+', Ag)
        
        # Compute log.lambda
        a <- Reduce(sum, Map('*', lapply(Ag, function(x) determinant(x)$modulus * determinant(x)$sign), (ng - 1)/2))
        b <- (n - G)/2 * determinant(A)$modulus * determinant(A)$sign
        c <- ((n - G) * p)/2 * log(n - G)
        d <- sum(p * (ng - 1)/2 * log(ng - 1))                            
        return(a - b + c - d) # log.lambda
    }

Now, we need the sampling distribution, under $H_0$, of $\ln(\lambda^{*}_1)$. To that end, let's define $\rho$ and $\omega_2$ as

$$\rho := 1-\frac{2p^{2}+3p-1}{6(p+1)(G-1)}\left(\left\{\sum_{g=1}^{G}\frac{1}{n_{g}-1} \right\} - \frac{1}{\left\{\sum_{g=1}^{G} n_{g}-1 \right\}}  \right),$$

and

$$\omega_{2}:= p(p+1)\left[(p-1)(p+2)\left(\left\{\sum_{g=1}^{G}\frac{1}{(n_{g}-1)^{2}} \right\} - \frac{1}{(n-G)^{2}} \right) - 6(G-1)(1-\rho)^{2} \right] \frac{1}{48\rho^{2}}.$$

The functions ```rho(Ag, ng, G, p, n)``` and ```omega2(Ag, ng, G, p, n)``` compute $\rho$ and $\omega_{2}$, respectively.

In [155]:
rho <- 
    function(Ag, ng, G, p, n){
                
        # Arguments:
        # Ag -- list containing each one of the Ag matrices for each sub-population
        # ng -- vector containing each one of the sample sizes for each sub-population
        # G -- total number of sub-populations
        # p -- dimension of the feature space
        # n -- overall sample size
    
        # Returns:
        # rho -- real value        
        
        # Compute rho
        s <- (((2 * p^2) + (3 * p) - 1)/(6 * (p + 1) * (G - 1))) * (sum(1/(ng - 1)) - (1/sum(ng - 1)))
        return(1 - s) # rho
    }

In [156]:
omega2 <- 
    function(Ag, ng, G, p, n){
        
        # Arguments:
        # Ag -- list containing each one of the Ag matrices for each sub-population
        # ng -- vector containing each one of the sample sizes for each sub-population
        # G -- total number of sub-populations
        # p -- dimension of the feature space
        # n -- overall sample size
    
        # Returns:
        # omega2 -- real value 
                
        # Compute omega2
        s1 <- (sum(1/(ng - 1)^(2)) - (1/(n - G)^(2)))
        s2 <- p * (p + 1) * ((p - 1) * (p + 2) * s1 - 6 * (G - 1) * (1 - rho(Ag, ng, G, p, n))^(2))
        return(s2/(48 * rho(Ag, ng, G, p, n)^(2))) # omega2
    }

Going back to the sampling distribution of $\ln(\lambda_{1}^{*})$, recall that each $\mathbf{x}^{g}_{i}$ is a $p$-dimensional normally distributed random vector, for $i\in\{1,\dots,\,n_{g}\}$ and $g\in\{1,\dots,\,G\}$. Therefore, each $A_g$ is a Wishart distributed random matrix with $(n_g - 1)$ degrees of freedom  and a positive definite scale matrix $\boldsymbol{\Sigma}_g$, this is

$$ A_g \sim \mathcal{W}(n_g -1,\, \boldsymbol{\Sigma}_g), \;\;\;\;\;\; g\in\{1,\dots,\,G\}.$$

When each $A_g$ is a Wishart distributed random matrix and when $\omega_2$ is small, it can be shown that

$$ -2\rho\ln(\lambda^{*}_1) \overset{aprox.}{\sim} \chi^{2}_{df}, $$

where $\chi^{2}_{df}$ denotes the Chi-square distribution with $df$ degrees of freedom. The degrees of freedom can be computed as

$$ df =  \frac{1}{2}(G-1)p(p+1).$$

The function ```df(G, p)``` computes $df$.

In [157]:
df <- 
    function(G, p){
        
        # Arguments:
        # G -- total number of sub-populations
        # p -- dimension of the feature space

        # Returns:
        # df -- real value 
        
        return((G - 1) * p * (p + 1)/2)   
    }   

Let's wrap up everything in a function called ```Box.M.test(Xg)```, where the argument ```Xg``` is a ```list``` containing each one of the data matrices $\mathbf{X}_{g}$, for $g\in\{1,\dots,\,G\}$.

In [158]:
Box.M.test <- 
    function(Xg){
        
        # Arguments:
        # Xg -- list containing each one of the design matrices for each sub-population
    
        # Returns: A list with the following components (similar to a 'htest' object)
        # statistic -- test statistic
        # omega2 -- omega_2
        # df -- degrees of freedom
        # pval -- p-value of the test
        
        G <- length(Xg) # number of sub-populations
        p <- dim(Xg[[1]])[2] # dimension of feature space
        ng <- sapply(Xg, function(x) dim(x)[1]) # sample sizes
        n <- sum(ng) # overall sample size
        
        # Compute Ag matrices
        Ag <- Map('*', lapply(Xg, function(x) cov(x)), (ng - 1))
        
        l.lambda <- log.lambda(Ag, ng, G, p, n) # log.lambda
        r <- rho(Ag, ng, G, p, n) # rho
        w <- round(omega2(Ag, ng, G, p, n), 3) # omega2
        d <- df(G, p) # degrees of freedom 
        
        statistic <- round(-2 * r * l.lambda, 2) # test statistic
        pval <- pchisq(statistic, d, lower.tail = FALSE) # p-value
    
        return(list('statistic' = as.numeric(statistic),
                    'omega2' = as.numeric(w),
                    'df' = as.numeric(d),
                    'pval' = as.numeric(pval)))
    }

Now, we are going to test our function on the well-known ```iris``` data-set. In this data-set, we have three sub-populations or groups: (1) Setosa, (2) Versicolor, and (3) Virginica, so $G = 3$. Moreover, for each observation, we have four numerical features, therefore $p = 4$. The sample size for each group is 50, i.e., $n_1 = n_2 = n_3 = 50$. 

In [159]:
Data <- iris

# Filter sub-populations and create Xg (Design matrices) for each group
Xg <- lapply(levels(Data$Species), function(x) Data[Data$Species == x,][,-5])

In [160]:
print(Box.M.test(Xg))

$statistic
[1] 140.94

$omega2
[1] 0.002

$df
[1] 20

$pval
[1] 3.356506e-20



We can observe that our implementation is working correctly.

## <a class="anchor" id="curse-dim"></a>The Curse of Dimensionality

Note, however, that in the previous example, the dimension of the feature space was $p=4$, which means that we were in a "low-dimensional" setting, where traditional multivariate statistical techniques work well. Nonetheless, in the era of *Big Data*, it is relatively easy to encounter data-sets with a large number of features, this is, $p\rightarrow \infty$. This introduces
a series of statistical challenges because standard statistical techniques were not designed to handle these types of problems; these challenges are often referred to as *the curse of dimensionality*.

As discussed in Johnson and Wichern (2007), when $n\, >\, 20$ and $p,\,G\, > \,5$, Box’s M-test does not work well. So naturally, the following question arises: What to do in high-dimensional and more complex settings? One possibility is to exploit recent advances in Random Matrix Theory.


## <a class="anchor" id="random-matrix"></a>Random Matrix Theory and Central Limit Theorems

Now, we will introduce some refined methodologies that take advantage of recent advances in Random Matrix Theory and Central Limit Theorems.

As before, we are going to consider the data matrices $\mathbf{X}_{g}$, for $g\in\{1,\dots,\,G\}$, along with $n$, $A_g$, and $A$, as defined before. We still want to test

$$H_0: \boldsymbol{\Sigma}_1 = \dots =\boldsymbol{\Sigma}_G = \boldsymbol{\Sigma}.$$

In Dette and Dörnemann (2020), the authors considered a modified likelihood ratio test statistic, as

$$\tilde{\Lambda}_{1} = \frac{\prod_{g=1}^{G}\left|\frac{A_{g}}{(n_{g}-1)}\right|^{\frac{(n_{g}-1)}{2}} }{\left| \frac{A}{(n - G)}\right|^{\frac{(n-G)}{2}}}.$$

Taking $\log$, we have that

$$\ln\left(\tilde{\Lambda}_{1}\right) = \left\{\sum_{g=1}^{G}\frac{(n_{g}-1)}{2}\ln\left( \begin{vmatrix} \frac{A_{g}}{(n_{g}-1)} \end{vmatrix}    \right)\right\} - \frac{(n-G)}{2}\ln\left( \begin{vmatrix} \frac{A}{(n-G)} \end{vmatrix} \right).$$

The function ```log.Lambda1(Ag, ng, G, n)``` computes $\ln\left(\tilde{\Lambda}_{1}\right)$. As before, the arguments of the function are ```Ag``` which is a ```list``` that consists of all $A_{g}$ matrices (for $g\in\{1,\dots,\,G\}$), ```ng``` which is a ```vector``` that contains the group sample sizes for each sub-population, ```G``` which is the total number of sub-populations, and ```n``` which is the overall sample size.

In [161]:
log.Lambda1 <- 
    function(Ag, ng, G, n){ 
        
        # Arguments:
        # Ag -- list containing each one of the Ag matrices for each sub-population
        # ng -- vector containing each one of the sample sizes for each sub-population
        # G -- total number of sub-populations
        # n -- overall sample size
    
        # Returns:
        # log.Lambda1 -- real value (as in Dette and Dornemann 2020)
        
        # Obtain matrix A
        A <- Reduce('+', Ag)
        
        # Compute log.Lambda1
        a <- Reduce(sum, Map('*', lapply(Map('/', Ag, (ng - 1)), function(x) determinant(x)$modulus * determinant(x)$sign), (ng - 1)/2))
        b <- (n - G)/2 * determinant(A/(n - G))$modulus * determinant(A/(n - G))$sign
        return(a - b) # log.Lambda1
    }

The next step is to obtain the asymptotic distribution of $\ln\left(\tilde{\Lambda}_{1}\right)$. Following Dette and Dörnemann (2020), we have that as $p,G \rightarrow \infty$ and under some technical conditions

$$ \frac{\ln\left(\tilde{\Lambda}_{1}\right) - \tilde{s_{n}}}{\sqrt{p(n-G)}} \sim \mathcal{N}(0, \sigma^{2}),$$

where

$$ \frac{1}{2}\sum_{g=1}^{G}\left\{ \frac{(n_{g}-1)^{2}}{p(n-G)}\ln\left(\frac{n_{g}-1}{n_{g}-p-1} \right)   \right\} - \frac{1}{2} \to \sigma^{2},$$

and

$$ \tilde{s_{n}} = \left[ \sum_{g=1}^{G} \frac{n_{g}-1}{2}\left\{ \left( n_{g}-\frac{3}{2}\right) \ln\left(\frac{n_{g}-2}{n_{g}-p-2} \right)  -p\ln\left( \frac{n_{g}-1}{n_{g}-p-2} \right) \right\} \right] - 
\frac{n-G}{2}(n-G-p)\ln\left( \frac{n-G}{n-G-p}  \right).$$

The functions ```sigma2(Ag, ng, G, p, n)``` and ```s.tilde(Ag, ng, G, p, n)``` compute $\sigma^{2}$ and $\tilde{s_{n}}$, respectively.

In [162]:
sigma2 <- 
    function(Ag, ng, G, p, n){

        # Arguments:
        # Ag -- list containing each one of the Ag matrices for each sub-population
        # ng -- vector containing each one of the sample sizes for each sub-population
        # G -- total number of sub-populations
        # p -- dimension of the feature space
        # n -- overall sample size
    
        # Returns:
        # sigma2 -- real value       
        
        return(((1/2) * sum((((ng - 1)^(2))/(p * (n - G))) * log((ng - 1)/(ng - p - 1)))) - (1/2))
    }

In [163]:
s.tilde <- 
    function(Ag, ng, G, p, n){
        
        # Arguments:
        # Ag -- list containing each one of the Ag matrices for each sub-population
        # ng -- vector containing each one of the sample sizes for each sub-population
        # G -- total number of sub-populations
        # p -- dimension of the feature space
        # n -- overall sample size
    
        # Returns:
        # s.tilde -- real value   
        
        # Compute s.tilde
        a <- sum(((ng - 1)/2) * ((ng - 3/2) * log((ng - 2)/(ng - p - 2)) - p * log((ng - 1)/(ng - p - 2))))
        b <- ((n - G)/2) * (n - G - p) * log((n - G)/(n - G - p))
        return(a - b)
    }

Let's wrap up everything in a function called ```Dette.Dor.test(Xg)```, which follows the test proposed by Dette and Dörnemann (2020). As before, the argument ```Xg``` is a ```list``` that contains all the data matrices $\mathbf{X}_{g}$, for $g\in\{1,\dots,\,G\}$.

In [164]:
Dette.Dor.test <- 
    function(Xg){
        
        # Arguments:
        # Xg -- list containing each one of the design matrices for each sub-population
    
        # Returns: A list with the following components (similar to a 'htest' object)
        # log.Lambda1 -- log.Lambda1
        # statistic -- test statistic (as in Dette and Dornemann 2020)
        # pval -- p-value of the test
        
        G <- length(Xg) # number of sub-populations
        p <- dim(Xg[[1]])[2] # dimension of feature space
        ng <- sapply(Xg, function(x) dim(x)[1]) # sample sizes
        n <- sum(ng) # overall sample size
                     
        # Compute Ag matrices
        Ag <- Map('*', lapply(Xg, function(x) cov(x)), (ng - 1))
        
        Lambda1 <- log.Lambda1(Ag, ng, G, n) # log.Lambda1
        s2 <- sigma2(Ag, ng, G, p, n) # sigma2
        s <- s.tilde(Ag, ng, G, p, n) # s.tilde
                              
        statistic <- (Lambda1 - s)/(sqrt(p * (n - G))) # test statistic
        pval <- 2 * pnorm(-abs(statistic), 0, s2) # p-value
        
        return(list('log.Lambda1' = as.numeric(round(Lambda1, 3)),
                    'statistic' = as.numeric(round(statistic, 3)),
                    'pval' = as.numeric(pval)))        
    }

Let's test our function using the ```iris``` data-set.

In [165]:
Data <- iris

# Filter sub-populations and create Xg (Design matrices) for each group
Xg <- lapply(levels(Data$Species), function(x) Data[Data$Species == x,][,-5])

In [166]:
print(Dette.Dor.test(Xg))

$log.Lambda1
[1] -73.332

$statistic
[1] -2.553

$pval
[1] 0



Note that we reached the same conclusion as in Box's M-test from above. However, the test proposed by Dette and Dörnemann (2020) was designed primarily for high-dimensional settings. Therefore, we are going to simulate a high-dimensional problem.

### Simulation Experiment

We are going to simulate a setting with $G=3$ sub-populations, the sample size for each sub-population is $n_g=1000$ and the dimension of the feature space is $p = 784$, for all $g\in\{1,\dots,\,G\}$.

All the observations across all sup-populations are drawn from a $p$-dimensional normal distribution with mean vectors

$$ \boldsymbol{\mu}_{g} = \mathbf{0}_{p}, \;\;\;\;\;\; g\in\{1,\dots,\,G\}, $$

where $\mathbf{0}_{p}$ denotes the $p$-dimensional zero vector. In other words, all the sub-populations share the same mean vector.

For the covariance matrices we are going to set $\boldsymbol{\Sigma}_{1} = \boldsymbol{\Sigma}_{2} = \boldsymbol{\Sigma}_{3} = \boldsymbol{\Sigma}$, with  

$$\boldsymbol{\Sigma}\sim\mathcal{IW}(p,\, \mathbf{I}_{p}), $$

where $\mathcal{IW}(p,\, \mathbf{I}_{p})$ denotes the Inverse-Wishart distribution with $p$ degrees of freedom and positive definite scale matrix $\mathbf{I}_{p}$. The matrix $\mathbf{I}_{p}$ denotes the identity matrix of size $p\times p$. This means that the three sub-populations were drawn from the same distribution with the same covariance matrices.

The function ```data.sim()``` simulates the data.

In [167]:
data.sim <- 
    function(){
        
        # Arguments:
    
        # Returns: 
        # Xg -- list with all the design matrices for each sub-population
        
        # Function to generate n IW distributed random matrices, stored in a list, each one with v degrees of freedom and scale matrix S
        rinvwish <- 
            function(n, v, S){
                lapply(1:n, function(x){Z <- matrix(rnorm(v * dim(S)[1]), v, dim(S)[1]) %*% chol(S); return(solve(t(Z) %*% Z))}) # solve() because is the IW distribution, not the W distribution.
            }


        # Function to generate n multivariate normal random vectors, stored as rows in a matrix, each one with mean vector Mu and covariance Sigma
        rmvnorm <-
            function(n, Mu, Sigma){
                t(t(matrix(rnorm(n * length(Mu)), n, length(Mu)) %*% chol(Sigma)) + Mu)
            }
        
        set.seed(1001) # for replication purposes
        
        G <- 3 # number of sub-populations
        p <- 784 # dimension of the feature space
        ng <- 1000 # sample size of each sub-population

        Mu <- rep(0, p) # Mean vector, common across all sub-populations
        Sigma <- rinvwish(1, p, diag(p))[[1]] # Covariance matrix, common across all sub-populations
        
        return(lapply(1:G, function(x) rmvnorm(ng, Mu, Sigma)))
    }

In [168]:
Xg <- data.sim()

Let's apply the test proposed by Dette and Dörnemann (2020) on the simulated data. We should not reject $H_0$.

In [169]:
print(Dette.Dor.test(Xg))

$log.Lambda1
[1] -511568.1

$statistic
[1] 0.231

$pval
[1] 0.6289533



For comparison, we also conducted Box's M-test on the simulated data.

In [170]:
print(Box.M.test(Xg))

$statistic
[1] 666046.2

$omega2
[1] 9674.298

$df
[1] 615440

$pval
[1] 0



We can observe that the test proposed by Dette and Dörnemann (2020) is working well, as there is not enough evidence to reject $H_{0}$, while Box's M-test is rejecting $H_{0}$ for any reasonable significance level, which clearly is a mistake.  Nonetheless, strictly speaking, the asymptotic distribution of the test statistic proposed by  Dette and Dörnemann (2020) holds when $p, G \rightarrow \infty$. In the above example, we saw that $p\rightarrow\infty$, but $G$ was a fixed number.

Another approach is the test proposed by Jiang and Yang (2013), where the authors consider the same $\ln\left(\tilde{\Lambda}_{1}\right)$ as in Dette and Dörnemann (2020), but in this case, the authors studied a different test statistic (with a different asymptotic distribution) for the cases when $G$ is a fixed number.

In fact, Jiang and Yang (2013) showed that for a fixed number $G$, the distribution of $\ln(\tilde{\Lambda}_{1})$, under the null hypothesis, is

$$ \frac{\ln(\tilde{\Lambda}_{1}) - \mu_{n} }{(n-G)\sigma_{n}^{(1)}} \overset{D}{\to} \mathcal{N}(0,1) \;\;\;\;\;\; \text{as} \;\;\;\;\;\; n_{g}, n, p \to \infty,$$


where 

$$ \mu_{n} = \frac{1}{4} \left\{(n-G)(2n-2p-2G-1)\ln\left( 1-\frac{p}{n-G}\right) - \sum_{g=1}^{G}(n_{g}-1)(2n_{g}-2p-3)\ln\left( 1-\frac{p}{n_{g}-1} \right)   \right\}, $$

and

$$ \left( \sigma_{n}^{(1)}\right)^{2} = \frac{1}{2} \left\{      \ln\left( 1-\frac{p}{n-G} \right)  -\sum_{g=1}^{G}\left(\frac{n_{g}-1}{n-G} \right)^{2}\ln\left( 1- \frac{p}{n_{g}-1} \right) \right\}.$$

The functions ```mu_n(Ag, ng, G, p, n)``` and ```sigma2_n(Ag, ng, G, p, n)``` compute $\mu_{n}$ and $\left( \sigma_{n}^{(1)}\right)^{2}$, respectively.

In [171]:
mu_n <- 
    function(Ag, ng, G, p, n){
    
        # Arguments:
        # Ag -- list containing each one of the Ag matrices for each sub-population
        # ng -- vector containing each one of the sample sizes for each sub-population
        # G -- total number of sub-populations
        # p -- dimension of the feature space
        # n -- overall sample size
    
        # Returns:
        # mu_n -- real value 
        
        # compute mu
        a <- (n - G) * (2 * n - 2 * p - 2 * G - 1) * log(1 - (p/(n - G)))
        b <- sum((ng - 1) * (2 * ng - 2 * p - 3) * log(1 - (p/(ng - 1))))
        return((1/4) * (a - b))
    }

In [172]:
sigma2_n <- 
    function(Ag, ng, G, p, n){
        
        # Arguments:
        # Ag -- list containing each one of the Ag matrices for each sub-population
        # ng -- vector containing each one of the sample sizes for each sub-population
        # G -- total number of sub-populations
        # p -- dimension of the feature space
        # n -- overall sample size
    
        # Returns:
        # sigma2_n -- real value 
        
        # compute sigma2
        a <- log(1 - (p/(n - G)))
        b <- sum(((ng - 1)/(n - G))^(2) * log(1 - (p/(ng - 1))))
        return((1/2) * (a - b))
    }

Let's wrap up everything in a function called ```Jiang.Yang.test(Xg)```, which follows the test proposed by Jiang and Yang (2013). As before, the argument ```Xg``` is a ```list``` containing each one of the data matrices $\mathbf{X}_{g}$, for $g\in\{1,\dots,\,G\}$.

In [173]:
Jiang.Yang.test <- 
    function(Xg){
        
        # Arguments:
        # Xg -- list containing each one of the design matrices for each sub-population
    
        # Returns: A list with the following components ('similar' to a htest object)
        # log.Lambda1 -- log.Lambda1
        # statistic -- test statistic (as in Jiang and Yang (2013))
        # pval -- p-value of the test
        
        G <- length(Xg) # number of sub-populations
        p <- dim(Xg[[1]])[2] # dimension of feature space
        ng <- sapply(Xg, function(x) dim(x)[1]) # sample sizes
        n <- sum(ng) # overall sample size
                     
        # Compute Ag matrices
        Ag <- Map('*', lapply(Xg, function(x) cov(x)), (ng - 1))
        
        Lambda1 <- log.Lambda1(Ag, ng, G, n) # log.Lambda1
        mu <- mu_n(Ag, ng, G, p, n) # mu_n
        sigma2 <- sigma2_n(Ag, ng, G, p, n) # sigma2_n
                              
        statistic <- (Lambda1 - mu)/((n - G) * sqrt(sigma2)) # test statistic
        pval <- 2 * pnorm(-abs(statistic)) # p-value
        
        return(list('log.Lambda1' = as.numeric(round(Lambda1, 3)),
                    'statistic' = as.numeric(round(statistic, 3)),
                    'pval' = as.numeric(pval)))        
    }

Let's apply the test proposed by Jiang and Yang (2013) on the simulated data from before. As expected, we should not reject $H_0$.

In [174]:
print(Jiang.Yang.test(Xg))

$log.Lambda1
[1] -511568.1

$statistic
[1] 0.132

$pval
[1] 0.8953773



We can observe that the test proposed by Jiang and Yang (2013) is also working correctly and produces a slightly larger p-value than the test proposed by Dette and Dörnemann (2020).

## <a class="anchor" id="con"></a>Conclusion

Throughout this notebook, we presented a discussion on how to test for equality between covariance matrices and why traditional statistical techniques are not adequate for this task in high-dimensional scenarios. Moreover, we also presented two recent approaches that make use of Random Matrix Theory and Central Limit Theorems to test for equality between covariance matrices even in high-dimensional settings.

# References

Anderson, T. W. (2003). *An Introduction to Multivariate Statistical Analysis*. John Wiley and Sons. New York, NY.

Dette, H. and Dörnemann, N. (2020). Likelihood ratio tests for many groups in high dimensions. *Journal of Multivariate Analysis*, 178:104605.

Jiang, T. and Yang, F. (2013). Central limit theorems for classical likelihood ratio tests for high-dimensional normal distributions. *The Annals of Statistics*, 41(4):2029–2074.

Johnson, R. A. and Wichern, D. W. (2007). *Applied Multivariate Statistical Analysis*. Pearson Prentice Hall. Upper Saddle River, NJ.